In [1]:
import pickle
import os
import inflection
import math
import datetime
import pandas as pd
import numpy  as np
from flask import Flask, request, Response
import requests

df_store = pd.read_csv('\\Users\\micro\\Desktop\\repos\\3_ciclo_intermediario\\1_ds_em_producao\\data\\store.csv', low_memory = False)
df = pd.read_csv('\\Users\\micro\\Desktop\\repos\\3_ciclo_intermediario\\1_ds_em_producao\\data\\test.csv')

In [2]:
# merge test dataset + store (perfomance das lojas + características das lojas)
df_test = pd.merge(df, df_store, how = 'left', on = 'Store')

# lojas escolhidas para testar as predições
# df_test = df_test[df_test['Store'].isin([20, 23, 22])]

# removendo do teste os dias fechados
df_test = df_test[df_test['Open'] != 0]
df_test = df_test[~df_test['Open'].isnull()] # ~: diferente
df_test = df_test.drop('Id', axis = 1)

In [3]:
# converte o dataframe em json para fazer a comunicação entre os sistemas
data = json.dumps(df_test.to_dict(orient = 'records'))

In [4]:
# chamada em API
url = 'https://rossmann-api-5z78.onrender.com/rossmann/predict'
header = {'Content-type': 'application/json'} 
data = data

r = requests.post(url, data = data, headers = header) # post: precisa enviar algum dado para requerir
print('Status Code {}'.format(r.status_code)) # para ver se a requisição é válida ou não 

Status Code 200


In [5]:
# pega o json da requisição e converte em dataframe
d1 = pd.DataFrame(r.json(), columns = r.json()[0].keys())
d1.head()

,store,day_of_week,date,open,promo,state_holiday,school_holiday,store_type,assortment,competition_distance,...,year,month,day,week_of_year,year_week,competition_since,competition_time_month,promo_since,promo_time_week,prediction
0,1,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,c,basic,1270.0,...,2015,9,17,38,2015-37,2008-09-01T00:00:00.000,85,2015-09-14T00:00:00.000,0,3556.217773
1,3,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,a,basic,14130.0,...,2015,9,17,38,2015-37,2006-12-01T00:00:00.000,107,2011-03-28T00:00:00.000,233,7151.637695
2,7,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,a,extended,24000.0,...,2015,9,17,38,2015-37,2013-04-01T00:00:00.000,29,2015-09-14T00:00:00.000,0,6503.599609
3,8,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,a,basic,7520.0,...,2015,9,17,38,2015-37,2014-10-01T00:00:00.000,11,2015-09-14T00:00:00.000,0,5600.644531
4,9,4,2015-09-17T00:00:00.000,1.0,1,regular_day,0,a,extended,2030.0,...,2015,9,17,38,2015-37,2000-08-01T00:00:00.000,184,2015-09-14T00:00:00.000,0,5676.948730


In [6]:
# agrupa as lojas e as predições
d2 = d1[['store', 'prediction']].groupby('store').sum().reset_index()

# mensagem mostrada ao usuário
# for i in range(len(d2)):
#     print('A loja {} irá vender R${:,.2f} nas próximas 6 semanas.'.format(d2.loc[i, 'store'], d2.loc[i, 'prediction']))

In [7]:
d2['prediction'] = np.round(d2['prediction'], 2)
d2

,store,prediction
0,1,149104.97
1,3,263219.79
2,7,281318.34
3,8,215527.73
4,9,247061.79
...,...,...
851,1111,205324.21
852,1112,338938.30
853,1113,240114.02
854,1114,707037.86


In [8]:
d2.to_csv('predictions.csv', index = False) 